In [1]:
import mujoco as mj
from mujoco.glfw import glfw
import numpy as np
import time as T

In [2]:


###
"""
data.qpos=q0; data.qpos[1]=q1 #set position
mj_forward(model,data) # forward kinematics
data.site_xpos[0] #get site position


"""
####
xml_path = 'left_hand_light.xml' #xml file (assumes this is in the same folder as this file)
simend = 20 # Simulation time

In [3]:

# MuJoCo data structures
model = mj.MjModel.from_xml_path(xml_path)  # MuJoCo model
data = mj.MjData(model)                # MuJoCo data

In [ ]:
# Get the number of joints
def getJointInfo(model):
    num_joints = model.njnt
    joint_info = []
    index=0
    for i in range(num_joints):
        # joint_name = model.name_jntadr[i]
        joint_index=index
        joint_body_id = model.jnt_bodyid[i]   
        index=index+1
        joint_type = model.jnt_type[i]
        joint_axis = model.jnt_axis[i] 
        joint_info.append((joint_index, joint_body_id, joint_type,joint_axis))
    return joint_info

In [ ]:
# move joint forward kinematics
for x in range(len(joint_info)):
    if joint_info[x][1]==9:
        print(data.qpos[x])
        print(x)

In [16]:
N = 500
simend=N
q0_start = 0 # starting angle
import math

def setPositionControl(joint_id,angle):
    q_start = 0
    q_end = math.radians(angle); # ending angle # 90 degrees-------------------
    q = np.linspace(q_start,q_end,N)
    data.qpos[joint_id] = q0_start;
    # print(q)
    return q
i = 0;
time = 0
dt = 0.001;
joint_move=[]
# for joint in range(16):
#     k=setPositionControl(joint,20)
#     joint_move.append(k)
#     print(len(k))
    # print(data.qpos)


In [15]:
# move finger by some anlge

joint_id=6
q=setPositionControl(joint_id,0)
for x in range(simend):
    step_start = T.time()
    mj.mj_step(model, data)
    joint_angles = np.copy(data.qpos)
    joint_torques = np.copy(data.qfrc_actuator)
    data.qpos[joint_id] = q[x];
    # data.qpos[1] = q1[i];

    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    print(f"angle {math.degrees(joint_angles[joint_id])}")
    
    # print(f"torque {joint_torques[joint_id]}")
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

    # print("Joint Angles:", joint_angles)
    # print("Joint Torques:", joint_torques)

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.06220514244549554
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.03710557042815856
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.028592874674957875
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.023370391745200468
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.019953739569992406
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.017707515210262502
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.01623034313116959
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.015259007625034845
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.014620403711477837
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle -0.0

In [ ]:
# print(len(data.xfrc_applied))

In [27]:
##### body_dict
import xml.etree.ElementTree as ET
xml_path="left_hand_light.xml"
tree = ET.parse(xml_path)
root = tree.getroot()
body_dict = {}
joint_dict={}
body_id=0
joint_flag=False
for body in root.findall('.//body'):
        body_name = body.get('name')
        if body_name is None:
            continue
        # body.findall('joint')
        for joint in body.findall('joint'):
            joint_flag=True
            joint_name = joint.get('name')
            # print(joint_name)
            body_dict[body_id]={"body_name":body_name,
                               "body_id":body_id,
                                "joint_name": joint.get('name')
            # "joint_type" : joint.get('type'),
            # # joint_pos = joint.get('pos'),
            # # joint_axis = joint.get('axis')
            # "joint_range" : joint.get('range')
            }
        if not joint_flag: # if link is present
            body_dict[body_id]={"body_name":body_name,
                               "body_id":body_id}
        joint_flag=False
            
        body_id+=1


In [28]:
print(body_dict)

{0: {'body_name': 'palm', 'body_id': 0}, 1: {'body_name': 'rf_base', 'body_id': 1, 'joint_name': 'rfj0'}, 2: {'body_name': 'rf_proximal', 'body_id': 2, 'joint_name': 'rfj1'}, 3: {'body_name': 'rf_medial', 'body_id': 3, 'joint_name': 'rfj2'}, 4: {'body_name': 'rf_distal', 'body_id': 4, 'joint_name': 'rfj3'}, 5: {'body_name': 'rf_tip', 'body_id': 5}, 6: {'body_name': 'mf_base', 'body_id': 6, 'joint_name': 'mfj0'}, 7: {'body_name': 'mf_proximal', 'body_id': 7, 'joint_name': 'mfj1'}, 8: {'body_name': 'mf_medial', 'body_id': 8, 'joint_name': 'mfj2'}, 9: {'body_name': 'mf_distal', 'body_id': 9, 'joint_name': 'mfj3'}, 10: {'body_name': 'mf_tip', 'body_id': 10}, 11: {'body_name': 'ff_base', 'body_id': 11, 'joint_name': 'ffj0'}, 12: {'body_name': 'ff_proximal', 'body_id': 12, 'joint_name': 'ffj1'}, 13: {'body_name': 'ff_medial', 'body_id': 13, 'joint_name': 'ffj2'}, 14: {'body_name': 'ff_distal', 'body_id': 14, 'joint_name': 'ffj3'}, 15: {'body_name': 'ff_tip', 'body_id': 15}, 16: {'body_name':

In [ ]:
# # Function to get actuator information
# def get_actuator_info(model):
#     actuators = []

#     for i in range(model.nu):
#         # actuator_name = model.mj_id2name(i, mujoco.mjtObj.mjOBJ_ACTUATOR)
#         actuator_trnid = model.actuator_trnid[i]
#         actuator_joint_id = actuator_trnid[0]  # Assuming the actuator controls a joint
#         # actuator_joint_name = model.id2name(actuator_joint_id, mujoco.mjtObj.mjOBJ_JOINT)

#         actuators.append({
#             'id': i,
#             # 'name': actuator_name,
#             'joint_id': actuator_joint_id,
#             # 'joint_name': actuator_joint_name,
#             'gear': model.actuator_gear[i],
#             'bias': model.actuator_biasprm[i]
#         })

#     return actuators

# # Get actuator information
# actuators = get_actuator_info(model)

In [20]:
# for body in root.findall('.//body'):
#         body_name = body.get('name')
#         if body_name is None:
#             continue
#         for joint in body.findall('ACTUATOR'):
#             joint_flag=True
#             joint_name = joint.get('name')
#             print(joint_name)

In [ ]:
# for actuator in root.findall('.//ACTUATOR'):
#     print(actuator)

In [ ]:
# print(actuator_info)

In [21]:
# # actuator name:[actuator_id, joint_id] 
# actuator_lsit=["ffa0":[0],"ffa1":[1],"ffa2":[2],"ffa3":[3],
#                "mfa0":[4],"mfa1":[5],"mfa2":[6],"mfa3":[7],
#                "rfa0":[8],"rfa1":[9],"rfa2":[10],"rfa3":[11],
#                "tha0":[12],"tha1":[13],"tha2":[14],"tha3":[15]
# ]

# def get_actuator_info(model):
#     actuators = []

#     for i in range(model.nu):
#         # actuator_name = model.mj_id2name(i, mujoco.mjtObj.mjOBJ_ACTUATOR)
#         actuator_trnid = model.actuator_trnid[i]
#         actuator_joint_id = actuator_trnid[0]  # Assuming the actuator controls a joint
#         # actuator_joint_name = model.id2name(actuator_joint_id, mujoco.mjtObj.mjOBJ_JOINT)
#         # print(model.nu)
#         actuators.append({
#             'id': i,
#             # 'name': actuator_name,
#             'joint_id': actuator_joint_id
#             # 'joint_name': actuator_joint_name,
#             # 'gear': model.actuator_gear[i],
#             # 'bias': model.actuator_biasprm[i]
#         })

#     return actuators

# # Get actuator information
# actuators = get_actuator_info(model)
# print(actuators)

In [29]:
# print(body_dict)
for key, value in enumerate(body_dict):
    if "joint_name" in body_dict[key]:
        # print(body_dict[key]["joint_name"])
        actuator_name=body_dict[key]["joint_name"].replace('j', 'a')
        body_dict[key]["actuator_name"]=actuator_name

In [30]:
print(body_dict)

{0: {'body_name': 'palm', 'body_id': 0}, 1: {'body_name': 'rf_base', 'body_id': 1, 'joint_name': 'rfj0', 'actuator_name': 'rfa0'}, 2: {'body_name': 'rf_proximal', 'body_id': 2, 'joint_name': 'rfj1', 'actuator_name': 'rfa1'}, 3: {'body_name': 'rf_medial', 'body_id': 3, 'joint_name': 'rfj2', 'actuator_name': 'rfa2'}, 4: {'body_name': 'rf_distal', 'body_id': 4, 'joint_name': 'rfj3', 'actuator_name': 'rfa3'}, 5: {'body_name': 'rf_tip', 'body_id': 5}, 6: {'body_name': 'mf_base', 'body_id': 6, 'joint_name': 'mfj0', 'actuator_name': 'mfa0'}, 7: {'body_name': 'mf_proximal', 'body_id': 7, 'joint_name': 'mfj1', 'actuator_name': 'mfa1'}, 8: {'body_name': 'mf_medial', 'body_id': 8, 'joint_name': 'mfj2', 'actuator_name': 'mfa2'}, 9: {'body_name': 'mf_distal', 'body_id': 9, 'joint_name': 'mfj3', 'actuator_name': 'mfa3'}, 10: {'body_name': 'mf_tip', 'body_id': 10}, 11: {'body_name': 'ff_base', 'body_id': 11, 'joint_name': 'ffj0', 'actuator_name': 'ffa0'}, 12: {'body_name': 'ff_proximal', 'body_id': 1

In [31]:

# [actuator name,actuator_id, joint_id] 

actuator_list=[["ffa0",0,8],["ffa1",1,9],["ffa2",2,10],["ffa3",3,11],
           ["mfa0",4,4],["mfa1",5,5],["mfa2",6,6],["mfa3",7,7],
        ["rfa0",8,0],["rfa1",9,1],["rfa2",10,2],["rfa3",11,3],
           ["tha0",12,12],["tha1",13,13],["tha2",14,14],["tha3",15,15]]
def actuator_key(act_name):

    # index_found=False
    for index in range(len(actuator_list)):
        if actuator_list[index][0]==act_name:
            # print(index)
            return index
        
for key,value in enumerate(body_dict):
    if 'actuator_name' in body_dict[key].keys():
        # print(body_dict[key]['actuator_name'])        
        ind=actuator_key(body_dict[key]['actuator_name'])
        print(actuator_list[ind])
        body_dict[key]['actuator_id']=actuator_list[ind][1]
        body_dict[key]['joint_id']=actuator_list[ind][2]

['rfa0', 8, 0]
['rfa1', 9, 1]
['rfa2', 10, 2]
['rfa3', 11, 3]
['mfa0', 4, 4]
['mfa1', 5, 5]
['mfa2', 6, 6]
['mfa3', 7, 7]
['ffa0', 0, 8]
['ffa1', 1, 9]
['ffa2', 2, 10]
['ffa3', 3, 11]
['tha0', 12, 12]
['tha1', 13, 13]
['tha2', 14, 14]
['tha3', 15, 15]


In [32]:
print(body_dict)

{0: {'body_name': 'palm', 'body_id': 0}, 1: {'body_name': 'rf_base', 'body_id': 1, 'joint_name': 'rfj0', 'actuator_name': 'rfa0', 'actuator_id': 8, 'joint_id': 0}, 2: {'body_name': 'rf_proximal', 'body_id': 2, 'joint_name': 'rfj1', 'actuator_name': 'rfa1', 'actuator_id': 9, 'joint_id': 1}, 3: {'body_name': 'rf_medial', 'body_id': 3, 'joint_name': 'rfj2', 'actuator_name': 'rfa2', 'actuator_id': 10, 'joint_id': 2}, 4: {'body_name': 'rf_distal', 'body_id': 4, 'joint_name': 'rfj3', 'actuator_name': 'rfa3', 'actuator_id': 11, 'joint_id': 3}, 5: {'body_name': 'rf_tip', 'body_id': 5}, 6: {'body_name': 'mf_base', 'body_id': 6, 'joint_name': 'mfj0', 'actuator_name': 'mfa0', 'actuator_id': 4, 'joint_id': 4}, 7: {'body_name': 'mf_proximal', 'body_id': 7, 'joint_name': 'mfj1', 'actuator_name': 'mfa1', 'actuator_id': 5, 'joint_id': 5}, 8: {'body_name': 'mf_medial', 'body_id': 8, 'joint_name': 'mfj2', 'actuator_name': 'mfa2', 'actuator_id': 6, 'joint_id': 6}, 9: {'body_name': 'mf_distal', 'body_id':

In [33]:
import json


# Save dictionary to JSON file
def save_dict_to_json(dictionary, filename):
    with open(filename, 'w') as json_file:
        json.dump(dictionary, json_file, indent=4)

# Load dictionary from JSON file
def load_dict_from_json(filename):
    with open(filename, 'r') as json_file:
        dictionary = json.load(json_file)
    return dictionary

# Save dict to a JSON file
save_dict_to_json(body_dict, 'body_dict.json')

# Load the dict frm JSON file
loaded_dict = load_dict_from_json('body_dict.json')

# loaded dict
print(loaded_dict)

{'0': {'body_name': 'palm', 'body_id': 0}, '1': {'body_name': 'rf_base', 'body_id': 1, 'joint_name': 'rfj0', 'actuator_name': 'rfa0', 'actuator_id': 8, 'joint_id': 0}, '2': {'body_name': 'rf_proximal', 'body_id': 2, 'joint_name': 'rfj1', 'actuator_name': 'rfa1', 'actuator_id': 9, 'joint_id': 1}, '3': {'body_name': 'rf_medial', 'body_id': 3, 'joint_name': 'rfj2', 'actuator_name': 'rfa2', 'actuator_id': 10, 'joint_id': 2}, '4': {'body_name': 'rf_distal', 'body_id': 4, 'joint_name': 'rfj3', 'actuator_name': 'rfa3', 'actuator_id': 11, 'joint_id': 3}, '5': {'body_name': 'rf_tip', 'body_id': 5}, '6': {'body_name': 'mf_base', 'body_id': 6, 'joint_name': 'mfj0', 'actuator_name': 'mfa0', 'actuator_id': 4, 'joint_id': 4}, '7': {'body_name': 'mf_proximal', 'body_id': 7, 'joint_name': 'mfj1', 'actuator_name': 'mfa1', 'actuator_id': 5, 'joint_id': 5}, '8': {'body_name': 'mf_medial', 'body_id': 8, 'joint_name': 'mfj2', 'actuator_name': 'mfa2', 'actuator_id': 6, 'joint_id': 6}, '9': {'body_name': 'mf

In [44]:
# import math

#initialize the controller
def set_position_servo(actuator_no, kp=10):
    bend_angle=20
    model.actuator_gainprm[actuator_no,0]=kp
    model.actuator_biasprm[actuator_no,1]=-kp
    data.ctrl=math.radians(bend_angle)

# target_actuator="ffa1"

def actuator_name2id(actuator_name):
    actuator_id=None
    for key, value in enumerate(body_dict):
         if "actuator_name" in body_dict[key]:
            if body_dict[key]["actuator_name"]==actuator_name:
            # actuator_name=body_dict[key]["joint_name"].replace('j', 'a')
                actuator_id=body_dict[key]["actuator_id"]
                return actuator_id





def controller(model,data):
    # spring like behaviour
    actuator_name="ffa1"
    actuator_id=actuator_name2id(actuator_name)
    set_position_servo(actuator_id,10)



#set the controller
mj.set_mjcb_control(controller)


def init_controller(model,data):
    pass





In [45]:
joint_id=13
for x in range(simend):
    step_start = T.time()
    mj.mj_step(model, data)
    joint_angles = np.copy(data.qpos)
    joint_torques = np.copy(data.qfrc_actuator)
    # data.qpos[6] = q0[x];
    # data.qpos[1] = q1[i];

    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    print(f"angle {math.degrees(joint_angles[joint_id])}")
    
    print(f"torque {joint_torques[joint_id]}")
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

    # print("Joint Angles:", joint_angles)
    # print("Joint Torques:", joint_torques)

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle 0.3960172896852643
torque 0.348977077438524
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle 0.7880318568427523
torque 0.34215404479903383
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle 1.1721943245557946
torque 0.33531209988635524
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle 1.5484432338440426
torque 0.32860719996217624
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle 1.9169355035054152
torque 0.32204041768805886
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle 2.277835621568313
torque 0.3156090143143211
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle 2.6313062221615615
torque 0.30931011898328725
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
angle 2.9775061230432804
torque 0.3031408931939333
XXXX